In [60]:
!pip install dspy-ai
import dspy

In [61]:
models=['Med-PaLM 2','mistral-7b-instruct','openchat-7b','gpt-3.5-turbo']
api_key = "API_KEY"

In [62]:
def initialize_model(model_name):
    return dspy.OpenAI(model=model_name, api_key=api_key)

no_of_models=len(models)
for i in range(no_of_models):
    current_model = initialize_model(models[i])
    dspy.settings.configure(lm=current_model)

In [63]:
!pip install datasets
from datasets import load_dataset

In [64]:
medqa_dataset = load_dataset("bigbio/med_qa")
print(medqa_dataset)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for bigbio/med_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/med_qa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 1273
    })
    validation: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 1272
    })
})


In [65]:
print(medqa_dataset["validation"][0])

{'meta_info': 'step1', 'question': 'A 21-year-old sexually active male complains of fever, pain during urination, and inflammation and pain in the right knee. A culture of the joint fluid shows a bacteria that does not ferment maltose and has no polysaccharide capsule. The physician orders antibiotic therapy for the patient. The mechanism of action of action of the medication given blocks cell wall synthesis, which of the following was given?', 'answer_idx': 'D', 'answer': 'Ceftriaxone', 'options': [{'key': 'A', 'value': 'Chloramphenicol'}, {'key': 'B', 'value': 'Gentamicin'}, {'key': 'C', 'value': 'Ciprofloxacin'}, {'key': 'D', 'value': 'Ceftriaxone'}, {'key': 'E', 'value': 'Trimethoprim'}]}


In [66]:
#PubMedQA
pubmedqa_dataset = load_dataset("bigbio/pubmed_qa")
print(pubmedqa_dataset)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for bigbio/pubmed_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/pubmed_qa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['QUESTION', 'CONTEXTS', 'LABELS', 'MESHES', 'YEAR', 'reasoning_required_pred', 'reasoning_free_pred', 'final_decision', 'LONG_ANSWER'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['QUESTION', 'CONTEXTS', 'LABELS', 'MESHES', 'YEAR', 'reasoning_required_pred', 'reasoning_free_pred', 'final_decision', 'LONG_ANSWER'],
        num_rows: 11269
    })
})


In [67]:
#MedMCQA
medmcqa_dataset = load_dataset("medmcqa")
print(medmcqa_dataset)

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 182822
    })
    test: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 6150
    })
    validation: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 4183
    })
})


In [83]:
#MMLU: clinical knowledge, medical genetics, anatomy, professional medicine, college biology, and college medicine. These are the datasets we will be using for our project.
knowledges = ['clinical_knowledge','medical_genetics','anatomy', 'professional_medicine', 'college_biology', 'college_medicine']
mmlu_dataset_professional = load_dataset("cais/mmlu",'professional_medicine')

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/272 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/31 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

In [84]:
medqa_dataset = load_dataset("bigbio/med_qa")

#training on the first 20 questions
train_subset = medqa_dataset["train"][:20]

train_questions = train_subset["question"]
train_answers_id = train_subset["answer_idx"]
train_options= train_subset["options"]
train_answer = train_subset["answer"]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for bigbio/med_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/med_qa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  download_mode=download_mode,


In [115]:
# # Data processing
formatted_train_options = [' '.join(f"{option['key']} {option['value']}" for option in options) for options in train_options]
print(formatted_train_options)

['A Ampicillin B Ceftriaxone C Ciprofloxacin D Doxycycline E Nitrofurantoin', 'A Placing the infant in a supine position on a firm mattress while sleeping B Routine postnatal electrocardiogram (ECG) C Keeping the infant covered and maintaining a high room temperature D Application of a device to maintain the sleeping position E Avoiding pacifier use during sleep', 'A Abnormal migration of ventral pancreatic bud B Complete failure of proximal duodenum to recanalize C Error in neural crest cell migration D Abnormal hypertrophy of the pylorus E Failure of lateral body folds to move ventrally and fuse in the midline', 'A Thromboembolism B Pulmonary ischemia C Pulmonary hypertension D Pulmonary passive congestion E Pulmonary hemorrhage', 'A Factor V Leiden B Hemophilia A C Lupus anticoagulant D Protein C deficiency E Von Willebrand disease', 'A Aspirin B Oral contraceptive pills C Scorpion sting D Hypothyroidism E Obesity', 'A Bilirubin assessment B Coagulation studies C Hematocrit assessme

In [85]:
# # Data processing
# formatted_train_options = []
# for option in train_options:
#   sublist=[]
#   for i in range(len(option)):
#     sublist.append({option[i]['key']: option[i]['value']})
#   formatted_train_options.append(sublist)

In [112]:
print(train_questions)
print(train_options)
print(train_answer[0])
print(train_answers_id[0])

['A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?', 'A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have prevented the death of the baby?', "A mother brings her 3-week-old infant to the pediatrician's office because she is concerned about his feeding habits. He was born without complic

In [120]:
# We are expecting a single choice answer so signature accordingly.
class MultipleChoiceQA(dspy.Signature):
    """Answer questions with single letter answers."""

    question = dspy.InputField(desc="The multiple-choice question.")
    options = dspy.InputField(desc="The set of options in the format : A option1 B option2 C option3 D option4 E option5")
    answer = dspy.OutputField(desc="A single-letter answer corresponding to the selected option.")

In [139]:
# Shuffling the questions to make the model better.
import random
generate_answer = dspy.Predict(MultipleChoiceQA)

def generate_responses(questions, option_sets):
    responses = []
    combined_data = list(zip(questions, option_sets))
    random.shuffle(combined_data)
    shuffled_questions, shuffled_option_sets = zip(*combined_data)

    for question, options_list in zip(shuffled_questions, shuffled_option_sets):
        pred_response = generate_answer(question=question, options=options_list)
        generated_response = pred_response.answer
        responses.append(generated_response)

    return responses

# def generate_responses(questions, option_sets):
#     responses = []
#     for question, options_list in zip(questions, option_sets):
#         pred_response = generate_answer(question=question, options=options_list)
#         generated_response = pred_response.answer
#         responses.append(generated_response)
#     return responses

In [126]:
def calculate_medqa_accuracy(pred, true):
    correct_count = 0

    for pred, ref in zip(pred, true):
        if pred.strip().upper() == ref.strip().upper():
            correct_count += 1

    accuracy = correct_count / len(pred)
    return accuracy

In [128]:
#Zero-shot(directly ask a question and get an answer no prior training.)
#Ignore error API_KEY not added right now.
epochs = 50

accuracy = []

for j in range(epochs):

  generated_responses = generate_responses(train_questions, formatted_train_options)
  running_accuracy = calculate_medqa_accuracy(generated_responses, train_answers_id)
  accuracy.append(running_accuracy)

  print('Training - Epoch {}/{}, loss:{:.3f} '.format(j+1, epochs, accuracy[j]))
  print('-------------------------------------------------------------------')

# Line Plot of accuracy vs epochs
import matplotlib.pyplot as plt

plt.plot(range(epochs),accuracy);plt.xlabel('Epochs');plt.ylabel('Accuracy')

AuthenticationError: ignored